In [1]:
import datajoint as dj
import pandas as pd
import numpy as np
import time

import matplotlib.pyplot as plt
import ipyvolume.pylab as p3

In [2]:
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')

Connecting cpapadop@10.28.0.34:3306


In [3]:
# fetched_mesh = (ta3p100.Mesh & ta3p100.CurrentSegmentation & 'segment_id=648518346341351503').fetch1()
# fetched_mesh = ta3p100.Decimation35.fetch(limit=1, as_dict=True)[0]
fetched_mesh = (ta3p100.Mesh & ta3p100.CurrentSegmentation & 'segment_id=648518346341366885').fetch1()

In [24]:
class Mesh:
    def __init__(self, vertices, triangles=None):
        self._vertices = vertices#.copy()
        self._triangles = triangles#.copy()
    
    class VoxelStruct:
        def __init__(self, origin, side_length, offset_vectors=None, voxel_vertices=None):
            """
            :param origin: The starting location of which the voxels will be offset from.
            :param side_length: The length of a side of a voxel (they are cubes).
            :param offset_vectors: The structure that stores the locations of the voxels as offset integers from the origin (using the side length as the increment).
            """
            self.origin = origin
            self.side_length = side_length
            if offset_vectors is not None:
                self.offset_vectors = offset_vectors
            if voxel_vertices is not None:
                self.voxel_vertices = voxel_vertices

        @property
        def origin(self):
            return np.array((self._x, self._y, self._z))

        @origin.setter
        def origin(self, coordinate):
            self._x, self._y, self._z = coordinate

        @property
        def side_length(self):
            return self._side_length

        @side_length.setter
        def side_length(self, side_length):
            self._side_length = side_length

        @property
        def offset_vectors(self):
            return self._offset_vectors

        @offset_vectors.setter
        def offset_vectors(self, offset_vectors):
            if offset_vectors.shape == (len(offset_vectors), 3):
                self._offset_vectors = offset_vectors
            else:
                raise TypeError("Array shape is incorrect, should be equivalent to (-1, 3).")
                
        @property
        def offset_to_bboxes(self):
            voxel_min = self.origin + (self.offset_vectors * self.side_length)
            voxel_max = voxel_min + self.side_length
            return np.stack([voxel_min, voxel_max], axis=2)
        
        @property
        def voxel_bboxes_to_drawable(self):
            return self.offset_to_bboxes[:, np.arange(3), self._rectangular_idx.T].transpose(0, 2, 1)

        # Maybe I should make this search_by_offset pretty extensible by some design? Like allow you to easily search by a bunch of x offsets,
        # or even all of them. And also still be able to search for a certain offset like (0, 37, 20). Basically be able to search for a row,
        # or from a column or a bunch of columns or for a bunch of rows, etc.

        #     def search_by_offset(self, x_offset, y_offset, z_offset):
        #         return np.where((vs.offset_vectors==(x_offset, y_offset, z_offset)).all(axis=1))[0]
        def search_by_offset(self, key):
            print(type(self.offset_vectors==key))
            return np.where((self.offset_vectors==key).all(axis=1))[0]
    
    @property
    def voxels(self):
        return self._voxels
    
    @voxels.setter
    def voxels(self, voxels):
#         if voxels is self.VoxelStruct:
        self._voxels = voxels
#         else:
#             raise TypeError("Wrong type yo, make it a VoxelStruct object.")
    
    @property
    def vertices(self):
        return self._vertices
        
    @staticmethod
    def get_bbox(vertices):
        return np.array([(np.min(axis), np.max(axis)) for axis in vertices.T])
        
    @property
    def bbox(self):
        return self.get_bbox(self.vertices)
    
    @property
    def _rectangular_idx(self):
        X = [0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
        Y = [0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0]
        Z = [0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0]
        return np.vstack((X, Y, Z))
    
    @property
    def voxels_to_bboxes(self):
        voxels = self.voxels
        voxel_min = voxels.origin + (voxels.offset_vectors * voxels.side_length)
        voxel_max = voxel_min + voxels.side_length
        return np.stack([voxel_min, voxel_max], axis=2)
    
    @property
    def voxel_bboxes_to_drawable(self):
        return self.voxels_to_bboxes[:, np.arange(3), self._rectangular_idx.T].transpose(0, 2, 1)
    
    @property
    def volume(self):
        """
        Returns the estimation of volume based on the voxelization. Units are the same as the vertex input.
        """
        return self.offset**3 * len(self.structure)
    
    # Could have a more configurable plotting function. Like you give the argument ['mesh', 'offset_vectors', 'mesh_bbox', etc.]
    def plot_voxels(self, voxel_count_offset=0, voxel_limit=None, use_centroids_instead=False, width=800, height=600):
        p3.figure(width=width, height=height)
        p3.plot_trisurf(*fetched_mesh['vertices'].T/1000, triangles=fetched_mesh['triangles'])#[:10000])
        if use_centroids_instead:
            centroids = mesh.structure_to_bboxes.mean(axis=2)
            if voxel_limit is not None:
                centroids = centroids[:1000]
            p3.scatter(*centroids.T/1000, color='blue', marker='sphere', size=0.25)
        else:
            # Make it so voxel_limit can be larger 
            bboxes = self.voxel_bboxes_to_drawable
            voxel_count = len(bboxes)
            if voxel_count_offset >= voxel_count:
                voxel_count_offset = voxel_count - 1
            if voxel_limit is not None:
                if voxel_limit < (voxel_count + voxel_count_offset):
                    bboxes = bboxes[voxel_count_offset:voxel_count_offset+voxel_limit]
                else:
                    bboxes = bboxes[voxel_count_offset:]
            for bbox in self.voxel_bboxes_to_drawable:
                p3.plot(*bbox/1000, color='blue')
        # Can make xyzlim stuck to the bboxes that are actually plotted.
        p3.squarelim()
        p3.show()
    
#     @staticmethod
#     def _sort_vertex_rows(vertices):
#         # This still might be slower.
#         a = vertices
#         a = a[a[:,2].argsort()] # First sort doesn't need to be stable.
#         a = a[a[:,1].argsort(kind='mergesort')]
#         a = a[a[:,0].argsort(kind='mergesort')]
#         return a
    
#     def create_voxels_struct_only(self, vertices, edges, sort_axis):
#         sorted_verts = vertices[vertices[:,sort_axis].argsort()]
#         offset_idx = np.unique(edges[1:].searchsorted(sorted_verts).T[sort_axis])
#         return offset_idx
    
    def apply_split(self, vertices, edges, sort_axis):
        """
        :param vertices: The vertices to sort through and split.
        :param edges: The edges along which to split the array.
        :param sort_axis: The axis to sort and split the array with.
        """
#         sorted_verts = vertices[vertices[:,sort_axis].argsort()]
#         splitter = sorted_verts[:,sort_axis].searchsorted(edges)
        
# #         uni, ind, cou = np.unique(splitter, return_index=True, return_counts=True)
# #         if cou[0] > 1:
# #             mask = np.where((ind>0)&(ind<len(splitter)))
# #             offset_idx = ind[mask]
# #             filtered = uni[mask]
# #         else:
# #             mask = np.where(ind<len(splitter))
# #             offset_idx = ind[mask]
# #             filtered = uni[np.where((ind>0)&(ind<len(splitter)))]
        
# #         offset_idx = np.where((splitter>0)&(splitter<len(sorted_verts)))[0]
# #         filtered = np.unique(splitter[offset_idx])
# #         if len(offset_idx) == len(filtered) + 2:
# #             offset_idx = offset_idx[1:]
# #         elif len(offset_idx) == len(filtered):
# #             offset_idx = np.insert(offset_idx, 0, 0)
        
# #         unique_split, offset_idx = np.unique(splitter, return_index=True)
# #         filtered = unique_split[(unique_split>0)&(unique_split<len(sorted_verts))]
# #         print(len(offset_idx), len(filtered))
# #         if len(offset_idx) == len(filtered) + 2:
# #             offset_idx = offset_idx[1:]
# #         elif len(filtered) == 0:
# #             offset_idx = np.delete(offset_idx, 0)
        
#         unique_split, offset_idx = np.unique(splitter, return_index=True)
#         filtered = unique_split[(unique_split>0)&(unique_split<len(sorted_verts))]
#         if len(offset_idx) == len(filtered) + 2:
#             offset_idx = offset_idx[1:]
        
#         return offset_idx, np.array(np.split(sorted_verts, filtered))
    
        sorted_verts = vertices[vertices[:,sort_axis].argsort()]
        splitter = sorted_verts[:,sort_axis].searchsorted(edges)
        split = np.array(np.split(sorted_verts, splitter)[1:])
        offset_idx = [i for i, block in enumerate(split) if len(block) > 0]
        # This commented out portion is actually slower, and doesn't seem to work if the voxel size is too small.
#         offset_idx = np.unique(edges[1:].searchsorted(sorted_verts).T[sort_axis])
        return np.array((offset_idx, split[offset_idx]))
                
    # Going to look to redo the CubeVoxelize method for speed and clarity.
    def voxelize(self, side_length): # Probably need to have the side_length only be set once. Maybe just have the Voxelization class do it all in the intialization.
        self._offset = side_length
        bbox = self.bbox
        # Get the number of voxels to be used to create cubes (allowing for pushing past the boundaries).
        num_voxels = np.ceil((np.abs(np.subtract(*bbox.T) / side_length))).astype(int)
        
        # Create the cube voxel grid split structure. Could also sort the vertices according to the grid at this point?
        start_coord = bbox.T[0]
        cube_friendly_bbox = np.vstack((start_coord, start_coord + (num_voxels * side_length))).T
#         cube_friendly_bbox = np.vstack((start_coord, start_coord + (num_voxels * side_length) + 1)).T
        x_edges, y_edges, z_edges = [np.arange(minimum, maximum, side_length) for minimum, maximum in cube_friendly_bbox]
        self.cube_friendly_bbox = cube_friendly_bbox # Don't actually need this as a class variable
        self.x_edges, self.y_edges, self.z_edges = x_edges, y_edges, z_edges # Don't actually need this as a class variable
#         print([len(x_edges), len(y_edges), len(z_edges)])        
        
        offset_vectors = list()
        voxel_vertices = dict()
        
        # HOW do I recover the voxel indexes though? Because that's all I care about... Do I just literally only use split and just keep indexing?
        # how do I deal with the getting the correct y_offset indices from inside an x_split?
        
        # Need to do the initial sort on the x_axis
        x_split = np.array(self.apply_split(self.vertices, x_edges, 0)).T
        for x_id, x_block in x_split:
            y_split = np.array(self.apply_split(x_block, y_edges, 1)).T
            
            for y_id, y_block in y_split:
                z_split = np.array(self.apply_split(y_block, z_edges, 2)).T

                for z_id, z_block in z_split:
                    key = (x_id, y_id, z_id)
                    offset_vectors.append(key)
                    voxel_vertices[key] = z_block

        offset_vectors = np.array(offset_vectors)
        self.voxels = self.VoxelStruct(self.bbox[:,0], side_length, offset_vectors, voxel_vertices)
        
        return offset_vectors, voxel_vertices

In [25]:
self = Mesh(fetched_mesh['vertices'])

In [29]:
%%time

vectors, vox_verts = self.voxelize(side_length=2500)
print(len(vectors))

2434
CPU times: user 250 ms, sys: 3.01 ms, total: 253 ms
Wall time: 251 ms


In [215]:
self.plot_voxels()

In [40]:
examp_1 = self.vertices, self.x_edges, 0
examp_2 = x_block, y_edges, 1
examp_3 = y_block, z_edges, 2

In [133]:
def apply_split(vertices, edges, sort_axis):
    sorted_verts = vertices[vertices[:,sort_axis].argsort()]
    offset_idx = np.unique(edges[1:].searchsorted(sorted_verts).T[sort_axis])
#     print(offset_idx)
    splitter = sorted_verts[:,sort_axis].searchsorted(edges[1:])
#     print(len(splitter), splitter)
    split = np.array(np.split(sorted_verts, splitter))#offset_idx))
#     print(split[offset_idx])
#     offset_idx = np.unique(edges.searchsorted(sorted_verts).T[sort_axis][1:]) - 1
#     print(offset_idx)
#     offset_idx = [i for i, block in enumerate(split) if len(block) > 0]
#     print(offset_idx)
#     print(split.shape)
    return np.array((offset_idx, split[offset_idx]))
#     np.where(split.)
#     return np.array([(i, block) for i, block in enumerate(split) if len(block) > 0])
    
#     # offset_idx = np.where((splitter>0)&(splitter<len(sorted_verts)))[0]
#     unique_split, offset_idx = np.unique(splitter, return_index=True)
#     filtered = unique_split[(unique_split>0)&(unique_split<len(sorted_verts))]
#     if len(offset_idx) == len(filtered) + 2:
#         offset_idx = offset_idx[:-1]
#     if len(filtered) == 0:
#         offset_idx[:] = len(splitter) - 1
#     split = np.array(np.split(sorted_verts, filtered))

#     offset_idx.shape, unique_split.shape, filtered.shape, split.shape

In [134]:
%%timeit

# side_length = 10000

# self._offset = side_length
# bbox = self.bbox
# # Get the number of voxels to be used to create cubes (allowing for pushing past the boundaries).
# num_voxels = np.ceil((np.abs(np.subtract(*bbox.T) / side_length))).astype(int)

# # Create the cube voxel grid split structure. Could also sort the vertices according to the grid at this point?
# start_coord = bbox.T[0]
# cube_friendly_bbox = np.vstack((start_coord, start_coord + (num_voxels * side_length))).T
# x_edges, y_edges, z_edges = [np.arange(minimum, maximum, side_length) for minimum, maximum in cube_friendly_bbox]

x_split = apply_split(*examp_1) # self.vertices, x_edges, 0

x_split[0]

64.2 ms ± 275 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [116]:
x_split.shape

(2, 3)

In [572]:
%%timeit

np.array(apply_split(*examp_1)).shape

30.1 ms ± 252 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [177]:
%%time

side_length = 2048

self._offset = side_length
bbox = self.bbox
# Get the number of voxels to be used to create cubes (allowing for pushing past the boundaries).
num_voxels = np.ceil((np.abs(np.subtract(*bbox.T) / side_length))).astype(int)

# Create the cube voxel grid split structure. Could also sort the vertices according to the grid at this point?
start_coord = bbox.T[0]
cube_friendly_bbox = np.vstack((start_coord, start_coord + (num_voxels * side_length))).T
x_edges, y_edges, z_edges = [np.arange(minimum, maximum, side_length) for minimum, maximum in cube_friendly_bbox]

offset_vectors = list()
offset_to_vertices = dict()

x_split = np.array(self.apply_split(self.vertices, x_edges, 0)).T

for x_id, x_block in x_split:
    y_split = np.array(self.apply_split(x_block, y_edges, 1)).T
    
    for y_id, y_block in y_split:
        z_split = np.array(self.apply_split(y_block, z_edges, 2)).T
        
        for z_id, z_block in z_split:
            key = (x_id, y_id, z_id)
            offset_vectors.append(key)
            offset_to_vertices[key] = z_block

offset_vectors = np.array(offset_vectors)

CPU times: user 278 ms, sys: 6.94 ms, total: 285 ms
Wall time: 283 ms
